In [1]:
from lab_tsp_insertion import *
import math

In [10]:
from functools import lru_cache
class TSPSolution(object):
    def __init__(self, instance: list, tour:list, distance:int):
        self.tour = tour
        self.n_cities = len(tour)
        self.inst = instance
        self.distance = self.calculate_solution_distance()
    
    def __repr__(self):
        return f"Tour={self.tour}, distance={self.distance}, n_cities={self.n_cities}"
    
    def get_n_cities(self):
        return self.n_cities
    
    @lru_cache(maxsize=1024 * 1024 * 1024)
    def get_distance(self, x, y):
        # This function takes the city number, not index into array
        c1x, c1y = self.inst[x]
        c2x, c2y = self.inst[y]
        return ((c2x - c1x) ** 2 + (c2y - c1y) ** 2)
    
    def copy(self):
        return TSPSolution(self.inst, self.tour.copy(), self.distance)
    
    @lru_cache(maxsize=1024 * 1024 * 1024)
    def is_valid_swap(self, x:int, y:int):
        x, y = min(x, y), max(x, y)
        x1 = (x + 1) % self.n_cities
        y1 = (y + 1) % self.n_cities
        return x != y and x != y1 and x != x1 and y != y1 and y != x1 and y1 != x1
    
    def calculate_cost_if_swapped(self, x:int, y: int):
        # If the edges are swapped what would be the cost
        x, y = min(x, y), max(x, y)
        x1 = (x + 1) % self.n_cities
        y1 = (y + 1) % self.n_cities
        xx1 = self.get_distance(self.tour[x], self.tour[x1])
        xy = self.get_distance(self.tour[x], self.tour[y])
        yy1 = self.get_distance(self.tour[y], self.tour[y1])
        x1y1 = self.get_distance(self.tour[x1], self.tour[y1])
        #cs = self.get_distance.cache_info()
        #print(cs.hits / (cs.hits + cs.misses))
        return self.distance - xx1 - yy1 + x1y1 + xy
    
    def calculate_solution_distance(self):
        dist = 0
        for x in range(self.n_cities):
            y = (x + 1) % self.n_cities
            dist += self.get_distance(self.tour[x], self.tour[y])
        return dist
    
    def perform_swap(self, x, y):
        x, y = min(x, y), max(x, y)
        new_distance = self.calculate_cost_if_swapped(x, y)
        x1 = (x + 1) % self.n_cities
        y1 = (y + 1) % self.n_cities
        assert(x1 < y)
        self.tour[x1], self.tour[y] = self.tour[y], self.tour[x1]
        i = x1 + 1
        j = y - 1
        while (i < j):
            self.tour[i], self.tour[j] = self.tour[j], self.tour[i]
            i += 1
            j -= 1
        #if int(self.calculate_solution_distance()) != int(new_distance):
        #    print(f"New distance should have been {new_distance} but is {self.calculate_solution_distance()}")
        self.distance = new_distance


In [14]:
class TSPHillClimbing(object):
    def __init__(self, inst:dict = None, max_sideways_moves:int=0):
        self.inst = inst
        self.ind = None
        self.g_best_solution = None
        self.g_best_distance = 99999999999999999999999999999999999999999
        self.current_iter_sols = []
        self.current_iter_dist = None
        self.max_sideways_moves = max_sideways_moves
        self.n_sideways_moves = 0
        self.g_best_distance = 999999999999999999999999999999999999999999
        self.g_best_sol = None
        if None != self.inst:
            self.ind = self.get_solution()
            self.update_best_g_instance(self.ind)

            
    def __repr__(self):
        return f"inst = {self.inst}\n\nindividual = {self.ind}\n\n" +\
             f"g_best_solution = {self.g_best_solution}\n\ng_best_distance = {self.g_best_distance}"
    
    def update_best_g_instance(self, instance: TSPSolution):
        self.g_best_solution = instance.copy()
        self.g_best_distance = instance.distance
        
    def get_solution(self):
        cities, distance = insertion_heuristic1(self.inst)
        return TSPSolution(self.inst, cities, distance)
    
    def check_improving_move(self):
        self.current_iter_dist = self.ind.distance
        self.current_iter_sols = []
        for i in range(self.ind.get_n_cities()):
            for j in range(i+2, self.ind.get_n_cities()):
                if not self.ind.is_valid_swap(i, j):
                    #print(f"{i}, {j} is not a valid swap")
                    pass
                else:
                    newcost = self.ind.calculate_cost_if_swapped(i, j)
                    if (int(newcost) == int(self.current_iter_dist)):
                        if int(self.ind.distance) == int(newcost):
                            print("adding sideways")
                        self.current_iter_sols.append(list(i, j))
                    elif int(newcost) < int(self.current_iter_dist):
                        self.current_iter_dist = newcost
                        self.current_iter_sols = [[i,j]]
    
    def iterate(self):
        self.n_sideways_moves = 0
        t1 = time.perf_counter()
        for i in range(100):
            update = False
            self.check_improving_move()
            if int(self.ind.distance) == int(self.current_iter_dist):
                self.n_sideways_moves += 1
                if (self.n_sideways_moves > 10):
                    break
                if (len(self.current_iter_sols) == 0):
                    break
                update = True
                print("sideways")
            if int(self.ind.distance) > int(self.current_iter_dist):
                self.n_sideways_moves = 0
                update = True
            if update:
                toswap = self.current_iter_sols[random.randint(0, len(self.current_iter_sols)-1)]
                x, y = tuple(toswap)
                self.ind.perform_swap(x, y)
                print(i, self.ind.distance)
            if ((i % 50 == 0 or i < 10) and i != 0):
                t2 = time.perf_counter()
                t2 = t2 - t1
                t2 = t2 / i
                print("Avg time ", t2)
        print('-' * 80)
        
    def restart_and_iterate(self, n_restarts:int=5):
        for i in range(n_restarts):
            self.ind = self.get_solution()
            self.update_best_g_instance(self.ind)
            self.iterate()
                

In [15]:
random.seed(5)
def main():
    inst = readInstance('TSPData/inst-14.tsp')
    tsp = TSPHillClimbing(inst)
    tsp.restart_and_iterate()
    print('-' * 80)
    return tsp

In [16]:
tsp = main()

0 1721566490544
1 1493356599036
Avg time  10.54849633899994
2 1302368084298
Avg time  6.678244186999791
3 1129933474950
Avg time  5.397171866333186
4 1040692786864
Avg time  4.793405600999904
5 966205021200
Avg time  4.4438728955999975
6 893960291492
Avg time  4.222922299666607
7 825235634678
Avg time  4.039699803428513
8 771573733138
Avg time  3.89987349137499
9 730539994034
Avg time  3.786591193999963
10 706556708138
11 687228384290
12 670153457446
13 650712904886
14 636225330254
15 624619984504
16 614441007112
17 602492008720
18 592341874258
19 583432788688
20 574809852348
21 567351983566
22 560902920960
23 554944080254
24 548438019212
25 542630302832
26 536932935708
27 532612495626
28 528498996516
29 524472029938
30 520469449950
31 516664007060
32 513147443900
33 510159152108
34 507207518520
35 504279446296
36 501541865490
37 498913083904
38 496292463086
39 493807367170
40 491455875466
41 489114930454
42 486836567912
43 484844735670
44 482876156548
45 480929098990
46 479087440630
4

10 692285608106
11 673370713696
12 657425884044
13 642408245624
14 627869563572
15 616995002052
16 606208325364
17 595663674994
18 587527140492
19 580732911250
20 574226850208
21 568242773924
22 562617616032
23 558169047084
24 552814352220
25 548393122816
26 544028752364
27 539784979744
28 535704196808
29 531677230230
30 527686306536
31 523800427146
32 519988935576
33 516463140798
34 510746703094
35 507576209180
36 504838628374
37 502164926138
38 498490067198
39 495877926762
40 493288467702
41 490961171994
42 488690007210
43 486232767862
44 484100131270
45 482071694804
46 479978516352
47 477969923424
48 476073315378
49 474217308618
50 472367105840
Avg time  2.98056122158001
51 470525447480
52 468690853354
53 466859829136
54 465051111540
55 463302584054
56 459009642394
57 454006412440
58 450211777492
59 448466300540
60 446747075444
61 445077889410
62 443416413382
63 441741401208
64 440139216490
65 438638121480
66 437142668518
67 435201009862
68 433800959424
69 432420179120
70 4310506129

In [ ]:
print(tsp)